We first install the latest version of the `transformers` library (provides access to pre-trained language models like BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet etc.) 

In [ ]:
!git clone https://github.com/huggingface/transformers.git
!pip3 install --upgrade ./transformers

Install `emoji`

In [ ]:
! pip install emoji

In [ ]:
import os
import random
import time
import datetime
import torch
import argparse
import numpy as np
import pandas as pd
from torch.nn import functional as F
from transformers import (get_linear_schedule_with_warmup,AdamW,AutoModel, AutoTokenizer,
                            AutoModelForSequenceClassification)
from torch.utils.data import (TensorDataset,DataLoader,
                             RandomSampler, SequentialSampler, Dataset)

We define some helper functions to handle formatting:

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
train_df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
train_df

We feed the text input to the BERTweet tokenizer. 

For each input,we construct the following:
* **input ids:** a sequence of integers identifying each input token to its index number in the PureBERT tokenizer vocabulary
* **attention mask:** a sequence of 1s and 0s, with 1s for all input tokens and 0s for all padding tokens

In [ ]:
def bert_encode(df, tokenizer):
    input_ids = []
    attention_masks = []
    for sent in df[["text"]].values:
        sent = sent.item()
        encoded_dict = tokenizer.encode_plus(
                            sent,                      
                            add_special_tokens = True, 
                            max_length = 128,           
                            pad_to_max_length = True,
                            truncation = True,
                            return_attention_mask = True,   
                            return_tensors = 'pt',    
                    )
           
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    inputs = {
    'input_word_ids': input_ids,
    'input_mask': attention_masks}

    return inputs

We construct the PyTorch DataLoader for the training and test dataset. Note that we do not perform any pre-processing at all (the BERTweet tokenizer takes care of handling URLS, emojis etc.)!

In [ ]:
def prepare_dataloaders(train_df,test_df,batch_size=8):
    # Load the AutoTokenizer with a normalization mode if the input Tweet is raw
    
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)
    
    tweet_train = bert_encode(train_df, tokenizer)
    tweet_train_labels = train_df.target.astype(int)
    
    tweet_test = bert_encode(test_df, tokenizer)

    input_ids, attention_masks = tweet_train.values()
    labels = torch.tensor(tweet_train_labels.values)
    train_dataset = TensorDataset(input_ids, attention_masks, labels)

    
    input_ids, attention_masks = tweet_test.values()
    test_dataset = TensorDataset(input_ids, attention_masks)

    
    train_dataloader = DataLoader(
                train_dataset,
                sampler = RandomSampler(train_dataset), 
                batch_size = batch_size 
            )


    test_dataloader = DataLoader(
                test_dataset, 
                sampler = SequentialSampler(test_dataset), 
                batch_size = batch_size
            )
    return train_dataloader, test_dataloader

In [ ]:
train_dataloader,test_dataloader = prepare_dataloaders(train_df, test_df)


## Experiments

Below we study the tokenization process (i.e. mapping the input text from the tweets to input_ids from tokenizer) 

In [ ]:
def test_encode(sentence):
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)

    encoded_dict = tokenizer.encode_plus(
                        sentence,                      
                        add_special_tokens = True, 
                        max_length = 128,           
                        pad_to_max_length = True,
                        truncation = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',    
                )
           
    return encoded_dict['input_ids']

In [ ]:
def test_decode(tokens):
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)
    return tokenizer.convert_ids_to_tokens(tokens)

In [ ]:
train_df.text[0]

In [ ]:
text_test = train_df.text[0]
text_preprocessed = test_encode(text_test)


print(f'Shape      : {text_preprocessed.shape}')
print(f'Word Ids   : {text_preprocessed[0, :128]}')
print(test_decode(text_preprocessed[0, :128]))

## Training

Define the model using AdamW optimizer and an initial learning rate of 5e-5 and set epsilon to 1e-8.

Passing a model to *model_to_load* allows you to load a pre-trained BERTweet model - the default BERTweet model is `vinai/bertweet-bas`).

In [ ]:
def prepare_model(model_class="vinai/bertweet-base",num_classes=2,model_to_load=None,total_steps=-1):



    model = AutoModelForSequenceClassification.from_pretrained(
        model_class,
        num_labels = num_classes,  
        output_attentions = False, 
        output_hidden_states = False,
    )

    optimizer = AdamW(model.parameters(),
                    lr = 5e-5,
                    eps = 1e-8
                    )
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, 
                                                num_training_steps = total_steps)

    if model_to_load is not None:
        try:
            model.roberta.load_state_dict(torch.load(model_to_load))
            print("LOADED MODEL")
        except:
            pass
    return model, optimizer, scheduler

In [ ]:
epochs = 5
total_steps = len(train_dataloader) * epochs

model, optimizer, scheduler = prepare_model("vinai/bertweet-base" ,num_classes=2, model_to_load=None, total_steps = total_steps)

This is the training loop - during training, we print out the loss.

In [ ]:
def train(model,optimizer,scheduler,train_dataloader,epochs):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    training_stats = []
    total_t0 = time.time()

    for epoch_i in range(0, epochs):

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')
        
        t0 = time.time()
        total_train_loss = 0
        model.train()
        for step, batch in enumerate(train_dataloader):
            if step % 40 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            model.zero_grad()        
            outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask, 
                                labels=b_labels)
            loss = outputs.loss
            logits = outputs.logits
            total_train_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
        avg_train_loss = total_train_loss / len(train_dataloader)            
        training_time = format_time(time.time() - t0)

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(training_time))

    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
train(model,optimizer,scheduler,train_dataloader, epochs)

## Inference

The `predict` method allows you to perform inference on the test set.

In [ ]:
def predict(model,test_dataloader):
    model.eval()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    preds = []

    for batch in test_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask)
            logits = outputs.logits

        logits = logits.detach().cpu().numpy()
        for logit in logits:
            preds.append(logit)

    return preds

In [ ]:
result = predict(model,test_dataloader)

In [ ]:
from scipy.special import softmax

pred_labels = np.argmax(result, axis = 1)
pred_scores = softmax(result, axis=1)[:, 1]

In [ ]:
pred_labels

In [ ]:
output = pd.DataFrame({'id':test_df.id,'target':pred_labels})
output

In [ ]:
output.to_csv('submission.csv',index=False)